<a href="https://colab.research.google.com/github/Gokul0Krishna/pytorchpractice/blob/master/3rdattempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/synthetic.zip"
!unzip "/content/washer.v1i.multiclass.zip"
!unzip "/content/test.zip"
!unzip "/content/washer.v4i.multiclass.zip"

Archive:  /content/synthetic.zip
   creating: synthetic/
   creating: synthetic/nut/
  inflating: synthetic/nut/AKM_M10_1_10.png  
  inflating: synthetic/nut/AKM_M10_1_14.png  
  inflating: synthetic/nut/AKM_M10_1_15.png  
  inflating: synthetic/nut/AKM_M10_1_19.png  
  inflating: synthetic/nut/AKM_M10_1_20.png  
  inflating: synthetic/nut/AKM_M10_1_4.png  
  inflating: synthetic/nut/AKM_M10_1_5.png  
  inflating: synthetic/nut/AKM_M10_1_9.png  
  inflating: synthetic/nut/AKM_M12_1_10.png  
  inflating: synthetic/nut/AKM_M12_1_14.png  
  inflating: synthetic/nut/AKM_M12_1_15.png  
  inflating: synthetic/nut/AKM_M12_1_19.png  
  inflating: synthetic/nut/AKM_M12_1_20.png  
  inflating: synthetic/nut/AKM_M12_1_4.png  
  inflating: synthetic/nut/AKM_M12_1_5.png  
  inflating: synthetic/nut/AKM_M12_1_9.png  
  inflating: synthetic/nut/AKM_M14_1_10.png  
  inflating: synthetic/nut/AKM_M14_1_14.png  
  inflating: synthetic/nut/AKM_M14_1_15.png  
  inflating: synthetic/nut/AKM_M14_1_19.png  
 

In [ ]:
import torch
from torchvision import models
from torch.utils.data import DataLoader,Dataset

In [ ]:
import os
l=['washer','nut']
realimages=[]
reallabels=[]
for i in os.listdir("/content/washer.v4i.multiclass"):
  for j in os.listdir("/content/washer.v4i.multiclass/"+i):
    if j.endswith(".jpg") == True:
      realimages.append("/content/washer.v4i.multiclass/"+i+"/"+j)
      reallabels.append("washer")

for i in os.listdir("/content/washer.v1i.multiclass"):
    if i.endswith(".txt") == False:
      for j in os.listdir("/content/washer.v1i.multiclass/"+i):
        if j.endswith(".jpg") == True:
          realimages.append("/content/washer.v1i.multiclass/"+i+"/"+j)
          reallabels.append("washer")
a=0
for i in os.listdir("/content/test"):
  for j in os.listdir("/content/test/"+i):
      if a>250:
          break
      realimages.append("/content/test/"+i+"/"+j)
      reallabels.append("nut")
      a+=1

In [ ]:
syntheticimages=[]
syntheticlabels=[]
for i in os.listdir("/content/synthetic"):
  for j in os.listdir("/content/synthetic/"+i):
      syntheticimages.append("/content/synthetic/"+i+"/"+j)
      syntheticlabels.append(i)

In [ ]:
import pandas as pd
dfs=pd.DataFrame({'X':syntheticimages,'Y':syntheticlabels})
dfs=pd.get_dummies(dfs,columns=['Y'],dtype=float)
dfs.columns = dfs.columns.str.replace("Y_", "")

dfr=pd.DataFrame({'X':realimages,'Y':reallabels})
dfr=pd.get_dummies(dfr,columns=['Y'],dtype=float)
dfr.columns = dfr.columns.str.replace("Y_", "")

In [ ]:
from PIL import Image
import numpy as np
class customdataset(Dataset):
    def __init__(self,x,y,transform=None):
        self.x=x
        self.y=y
        self.transform=transform

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        imgpath=self.x.iloc[index]
        if self.transform:
            image=Image.open(imgpath).convert("RGB")
            image=self.transform(image)
        label_row = self.y.iloc[index, 0:2]
        label_tensor = torch.tensor(label_row.values.astype(np.float32))
        label_index = torch.argmax(label_tensor).item()

        return image,label_index

In [ ]:
from torchvision import transforms

synthetic_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [ ]:
real_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
from sklearn.model_selection import train_test_split
xstrain, xstest,ystrain,ystest = train_test_split(dfs['X'],dfs[l],random_state=42, test_size=0.2)

In [ ]:

traindataset=customdataset(x=xstrain,y=ystrain,transform=synthetic_transforms)
valdataset=customdataset(x=xstest,y=ystest,transform=synthetic_transforms)

trainloader=DataLoader(traindataset,batch_size=40,shuffle=True)
valloader=DataLoader(valdataset,batch_size=40,shuffle=True)

In [ ]:
testmodel = models.densenet121(pretrained=False)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
testmodel.classifier = torch.nn.Linear(testmodel.classifier.in_features, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(testmodel.parameters(), lr=0.0001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
testmodel = testmodel.to(device)

In [ ]:
num_epochs = 2

for epoch in range(num_epochs):
    testmodel.train()
    trunningloss,vrunningloss=0.0,0.0
    ttotal,vtotal = 0,0
    tacc,vacc=0.0,0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = testmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        trunningloss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class
        ttotal += labels.size(0)
        tacc += (predicted == labels).sum().item()


    testmodel.eval()
    with torch.no_grad():
      for inputs, labels in valloader:
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = testmodel(inputs)
          loss = criterion(outputs, labels)
          vrunningloss += loss.item()

          _, predicted = torch.max(outputs.data, 1)  # Get predicted class
          vtotal += labels.size(0)
          vacc += (predicted == labels).sum().item()

    print("="*20)
    print(f"""Epoch {epoch+1}
          train
          Loss: {trunningloss/len(trainloader)}
          Accuracy: {tacc*100/ttotal}
          val
          Loss: {vrunningloss/len(valloader)}
          Accuracy: {vacc*100/vtotal}
          """)

Epoch 1
          train
          Loss: 0.17507150931308021
          Accuracy: 94.61588969139855
          val
          Loss: 0.15525817982852458
          Accuracy: 92.6509186351706
          
Epoch 2
          train
          Loss: 0.07046609833933316
          Accuracy: 97.53775443204202
          val
          Loss: 0.03527506170794368
          Accuracy: 98.9501312335958
          
Epoch 3
          train
          Loss: 0.04328973612796467
          Accuracy: 99.11359159553513
          val
          Loss: 0.02189332409761846
          Accuracy: 99.21259842519684
          


In [ ]:
xtrain, xtest,ytrain,ytest = train_test_split(dfr['X'],dfr[l],random_state=42, test_size=0.2)
xtest, xval,ytest,yval = train_test_split(xtest,ytest,random_state=42, test_size=0.5)

In [ ]:

traindataset=customdataset(x=xtrain,y=ytrain,transform=real_transforms)
testdataset=customdataset(x=xtest,y=ytest,transform=real_transforms)
valdataset=customdataset(x=xval,y=yval,transform=real_transforms)

trainloader=DataLoader(traindataset,batch_size=10,shuffle=True)
testloader=DataLoader(testdataset,batch_size=10,shuffle=True)
valloader=DataLoader(valdataset,batch_size=10,shuffle=True)

In [ ]:
for param in testmodel.features.parameters():
    param.requires_grad = False
optimizer = torch.optim.Adam(testmodel.classifier.parameters(), lr=1e-4)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    testmodel.train()
    trunningloss,vrunningloss=0.0,0.0
    ttotal,vtotal = 0,0
    tacc,vacc=0.0,0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = testmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        trunningloss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class
        ttotal += labels.size(0)
        tacc += (predicted == labels).sum().item()


    testmodel.eval()
    with torch.no_grad():
      for inputs, labels in valloader:
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()
          outputs = testmodel(inputs)
          loss = criterion(outputs, labels)
          vrunningloss += loss.item()

          _, predicted = torch.max(outputs.data, 1)  # Get predicted class
          vtotal += labels.size(0)
          vacc += (predicted == labels).sum().item()

    print("="*20)
    print(f"""Epoch {epoch+1}
          train
          Loss: {trunningloss/len(trainloader)}
          Accuracy: {tacc*100/ttotal}
          val
          Loss: {vrunningloss/len(valloader)}
          Accuracy: {vacc*100/vtotal}
          """)

Epoch 1
          train
          Loss: 0.4641287337716033
          Accuracy: 81.04738154613466
          val
          Loss: 0.7021904786427816
          Accuracy: 68.62745098039215
          
Epoch 2
          train
          Loss: 0.464847652287018
          Accuracy: 80.54862842892769
          val
          Loss: 0.9268389791250229
          Accuracy: 74.50980392156863
          


In [ ]:
terunningloss,teacc=0.0,0.0
tetotal=0

testmodel.eval()
with torch.no_grad():
  for inputs, labels in testloader:
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = testmodel(inputs)
      loss = criterion(outputs, labels)
      terunningloss += loss.item()

      _, predicted = torch.max(outputs.data, 1)  # Get predicted class
      tetotal += labels.size(0)
      teacc += (predicted == labels).sum().item()

print("="*20)
print(f"""
      train
      Loss: {terunningloss/len(testloader)}
      Accuracy: {teacc*100/tetotal}
      """)


      train
      Loss: 0.5430489480495453
      Accuracy: 78.0
      
